# Desafio - Pedro Zottolo

Vamos utilizar dos mesmos recursos demonstrados no Git(https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb) e vejamos o resultado da classificação desta Net. 

Primeiro, vamos importar as bibliotecas necessárias:

In [4]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

Importar a base de dados do MNIST para a classificação

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
Path.BASE_PATH = path

Neste momento vemos quais "pastas" possuem no path, iremos seleciona-las mais tarde para os dados de treino e validação

In [5]:
path.ls()

(#2) [Path('testing'),Path('training')]

In [6]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/1'),Path('training/2'),Path('training/3'),Path('training/4'),Path('training/5'),Path('training/6'),Path('training/7'),Path('training/8'),Path('training/9')]

Para este problema queremos classificar apenas os dígitos 0 e 5, ou seja, dentre as pastas mostradas acima selecionaremos os paths correspondentes para o aprendizado de máquina:

In [7]:
zero = (path/'training'/'0').ls().sorted()
cinco = (path/'training'/'5').ls().sorted()

Agora, iremos criar os tensores das nossas imagens, contendo todos os zeros e cincos em cada um dos tensores,

In [8]:
tensores_z = [tensor(Image.open(o)) for o in zero]
tensores_c = [tensor(Image.open(o)) for o in cinco]
len(tensores_z)

5923

Precisaremos normalizar os dados de cada imagem, isto significa, como os valores estão entre 0 e 1 precisamos dividir os pixels pelo valor máximo deles, neste caso 255, sendo assim:


In [9]:
stacked_z = torch.stack(tensores_z).float()/255
stacked_c = torch.stack(tensores_c).float()/255

## Dados de treino

Feita a parte de divisão e seleceção de dados para o nosso problema, iremos criar nossas variáveis para o aprendizado, isto é, definir nossas variáveis de treino e de validação: train_x, train_y, valid_x e valid_y.

#### Variáveis de treino:

In [10]:
train_x = torch.cat([stacked_z, stacked_c]).view(-1, 28*28)

In [11]:
train_y = tensor([1]*len(zero) + [0]*len(cinco)).unsqueeze(1)
train_x.shape,train_y.shape

(torch.Size([11344, 784]), torch.Size([11344, 1]))

Neste momento, identificaremos os zeros com 1 e os cincos como 0.

#### Variáveis de validação:

In [12]:
z_tens_v = torch.stack([tensor(Image.open(o)) for o in (path/'testing'/'0').ls()])
z_tens_v = z_tens_v.float()/255
c_tens_v = torch.stack([tensor(Image.open(o)) for o in (path/'testing'/'5').ls()])
c_tens_v = c_tens_v.float()/255

In [13]:
valid_x = torch.cat([z_tens_v, c_tens_v]).view(-1, 28*28)
valid_y = tensor([1]*len(z_tens_v) + [0]*len(c_tens_v)).unsqueeze(1)

Precisamos criar um DataLoader de cada Dataset, tanto do treino quanto da validação

In [14]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
valid_dset = list(zip(valid_x,valid_y))

In [15]:
dl = DataLoader(dset, batch_size=256)

In [16]:
valid_dl = DataLoader(valid_dset, batch_size=256)

In [17]:
dls = DataLoaders(dl, valid_dl)
xb,yb = first(dl)

#### Parâmetros de inicialização e Funções

Para que o nosso modelo, queremos ter parâmetros aleatórios para a computação dos dados e funções que ajudem no aprendizados, ou seja, funções que sejam ativadas dado algum parâmetro de inicialização.

Primeiramente, vamos definir nossa função que irá inicializar nossos parâmetros de forma aleatória:

In [18]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

Agora nossas funções de ativação:

In [19]:
def linear1(xb): return xb@weights + bias

In [20]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [21]:
def sigmoid(x): return 1/(1+torch.exp(-x))

In [22]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

Nosso modelo de aprendizado será uma Net simples, dado que Neutral Networks possuem taxa de erros baixas [1] e que o nosso problema será de classificação de dois algarismos, podemos definir um modelo mais simples e ir vendo suas fraquezas e como melhorar

In [23]:
def simple_net(xb): 
    res = xb@w1 + b1
    res = res.max(tensor(0.0))
    res = res@w2 + b2
    return res

In [24]:
w1 = init_params((28*28,30))
b1 = init_params(30)
w2 = init_params((30,1))
b2 = init_params(1)

Então o Simple Net será dado por

In [25]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [26]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

Como queremos melhorar sempre a nossa perfomance de aprendizado, queremos que nosso programa aprenda de maneira otimizada também, para isso selecionaremos a melhor taxa de Learning Error, para isso escolhemos lr = 0.1[2]. Precisamos também otimizar o número de Epoch, como mostrado em [3] para esse problema o número de Epoch é bom entre 30 e 100, como estamos trabalhando com um Dataset grande, na grandeza de 6000 dados, e com uma Loss Function diferente da que é mostrada podemos escolher epoch = 50.

In [27]:
learn.fit(50, 0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.360193,0.502902,0.476496,00:01
1,0.221305,0.450967,0.497329,00:00
2,0.136942,0.316969,0.682158,00:00
3,0.088143,0.207040,0.822650,00:00
4,0.061772,0.141028,0.899038,00:00
5,0.047329,0.102775,0.927885,00:00
6,0.039047,0.079677,0.946581,00:00
7,0.034028,0.064690,0.956731,00:00
8,0.030786,0.054418,0.963675,00:00
9,0.028552,0.047091,0.968483,00:00


## Resultado

Vemos que a net simples desempenhou um bom resultado, 98,82%,  na classificação de 0 e 5. Alguns passos foram modificados e estudados a partir das referêcias abaixo, para atingir o melhor resultado possível, outras funções de Loss ou de Otimizer podiam ser diferentes como Adabound [4], como também outros modelos a serem tratados Extended Minimal e LeNet5 [5], dentre outros [6]. 


## Referências

[1] Baldominos, A.; Saez, Y.; Isasi, P. A Survey of Handwritten Character Recognition with MNIST and EMNIST. Appl. Sci. 2019, 9, 3169. https://doi.org/10.3390/app9153169
[2] Smith, L. Cyclical Learning Rates for Training Neural Networks. arXiv:1506.01186 [cs.CV]
[3] Afaq, S.; Rao, S. Significance Of Epochs On Training A Neural Network. International Journal of Scientific & Technology Research Volume 9, Issue 06, June 2020
[4] Luo, L.; Xiong, Y.; Liu, Y.; Sun, X. Adaptative Gradient Methods with Dynamic Bound of Learning Rate. ICLR 2019
[5] Teow, M. Understanding Convolutional Neural Networks Using A Minimal Model for Handwritten Digit Recognition. 978-1-5386-0846-3/17/31.00 ©2017 IEEE
[6] McDonnell, M.; Tissera, M.; Vladusich, T.; van Schaik, A.; Tapson, J. Fast, simple and accurate handwritten digit classification by training shallow neural network classifiers with the 'extreme learning machine' algorithm. arXiv:1412.8307v2 [cs.NE] 22 Jul 2015

